In [ ]:
# reach spectra_dereplication folder
sub_dir_spec = input_dir + "/" + entry + "/spectral_dereplication/"
# reach SIRIUS results
sub_dir_sir = input_dir + "/" + entry + "/insilico/SIRIUS/"
# new line
if os.path.exists(sub_dir_spec) and os.path.exists(sub_dir_sir):

    # list of all csv files in the spectral dereplication foler
    spec_msp_csv = glob.glob(
        input_dir + "/" + entry + "/spectral_dereplication" + "/*.csv"
    )
    # Sirius csv result file
    sir_msp_csv = input_dir + "/" + entry + "/insilico/MS1DATA.csv"

    # if both exist; which should be the case, even in case of 0 results
    if os.path.exists(sir_msp_csv) and os.path.exists(spec_msp_csv[0]):

        # read both csv files
        spec_msv = pd.read_csv(spec_msp_csv[0])
        sir_msv = pd.read_csv(sir_msp_csv)
        
        spec_msv = spec_msv[
            [
                "id_X",
                "premz",
                "rtmin",
                "rtmax",
                "rtmed",
                "rtmean",
                "col_eng",
                "pol",
                "int",
                "source_file",
                "mbank_results_csv",
            ]
        ]
        sir_msv = sir_msv[
            [
                "id_X",
                "premz",
                "rtmed",
                "rtmean",
                "int",
                "col_eng",
                "pol",
                "ms2Peaks",
                "ms1Peaks",
                "sirius_result_dir",
            ]
        ]

        merged_df = sir_msv.merge(
            spec_msv,
            how="inner",
            left_on=["premz", "rtmed", "rtmean", "int", "col_eng", "pol"],
            right_on=["premz", "rtmed", "rtmean", "int", "col_eng", "pol"],
        )
        merged_df["Formula"] = np.nan
        merged_df["SMILES"] = np.nan
        merged_df["PubChemID"] = np.nan
        merged_df["IUPAC"] = np.nan
        merged_df["synonyms"] = np.nan
        merged_df["AnnotationSources"] = np.nan
        merged_df["AnnotationCount"] = np.nan
        merged_df["MSILevel"] = np.nan
        merged_df["MCSS"] = np.nan
        merged_df["most specific class"] = np.nan
        merged_df["level 5"] = np.nan
        merged_df["subclass"] = np.nan
        merged_df["class"] = np.nan
        merged_df["superclass"] = np.nan
        merged_df["ClassificationSource"] = np.nan

        for_only_formula = []
        for_formula_canopus = []

        can_selec_dir = input_dir + "/" + entry + "/Candidate_Selection"
        if not os.path.isdir(can_selec_dir):
            os.mkdir(can_selec_dir)


In [ ]:
for mer, rows in merged_df.iterrows():
    print(merged_df["premz"][mer])
    if not isNaN(merged_df["sirius_result_dir"][mer]):
        sirius_csv = merged_df["sirius_result_dir"][mer].replace(
            "./", input_dir + "/"
        )
        #print(sirius_csv)
    else:
        df = pd. DataFrame(list())
        df. to_csv(input_dir + '/empty_csv.csv')
        sirius_csv = input_dir + '/empty_csv.csv'

    mbank_csv = merged_df["mbank_results_csv"][mer].replace(
        "./", input_dir + "/"
    )
    gnps_csv = (
        merged_df["mbank_results_csv"][mer]
        .replace("./", input_dir + "/")
        .replace("mbank", "gnps")
        .replace("MassBank", "GNPS")
    )
    hmdb_csv = (
        merged_df["mbank_results_csv"][mer]
        .replace("./", input_dir + "/")
        .replace("mbank", "hmdb")
        .replace("MassBank", "HMDB")
    )

    if (
        os.path.exists(sirius_csv)
        and os.path.exists(gnps_csv)
        and os.path.exists(mbank_csv)
        and os.path.exists(hmdb_csv)
    ):

        sirius_df = pd.read_csv(sirius_csv)
        if "formula" in sirius_csv:

            if len(sirius_df) > 0:

                if "smiles" not in sirius_df.columns and "molecularFormula" in sirius_df.columns:

                    #merged_df.loc[mer, "Formula"] = sirius_df["molecularFormula"][0]
                    #merged_df["AnnotationSources"][mer] = "SIRIUS-Formula"    

                    index = mer
                    Formula = sirius_df["molecularFormula"][0]
                    for_only_formula.append(
                        {
                            "index": index,
                            "Formula": Formula,
                            "AnnotationSources": "SIRIUS-Formula"
                        }
                    )

                    if "class" in sirius_df.columns:
                        #merged_df.loc[mer, "superclass"] = sirius_df["superclass"][0]
                        #merged_df.loc[mer, "class"] = sirius_df["class"][0]
                        #merged_df.loc[mer, "subclass"] = sirius_df["subclass"][0]
                        #merged_df.loc[mer, "ClassificationSource"] = "CANOPUS"
                        #merged_df.loc[mer, "AnnotationSources"] = "SIRIUS-Formula|CANOPUS"
                        index = mer
                        Formula = sirius_df["molecularFormula"][0]
                        most_specific_class = sirius_df["most specific class"][0]
                        level_5 = sirius_df["level 5"][0]
                        superclass = sirius_df["superclass"][0]
                        classes = sirius_df["class"][0]
                        subclass = sirius_df["subclass"][0]
                        for_formula_canopus.append(
                            {
                                "index": index,
                                "Formula": Formula,
                                "most specific class": most_specific_class,
                                "level 5": level_5,
                                "subclass": subclass,
                                "class": classes,
                                "superclass": superclass,
                                "ClassificationSource": "CANOPUS",
                                "AnnotationSources": "SIRIUS-Formula|CANOPUS"
                            }
                        )

                    sirius_df = []

        if "structure" in sirius_csv:

            if len(sirius_df) > 0:

                if len(sirius_df) > 50:
                    sirius_df = sirius_df[0:50]
                if "smiles" in sirius_df.columns:

                    sirius_df = sirius_df.drop_duplicates("smiles")
                    sirius_df = sirius_df.dropna(subset=["smiles"])
                    merged_df["Formula"][mer] = sirius_df["molecularFormula"][0]
                    merged_df.loc[mer, "PubChemID"] = sirius_df["pubchemids"][0]
                    if "class" in sirius_df.columns:
                        if "most specific class" in sirius_df.columns and "level 5" in sirius_df.columns:
                            merged_df["most specific class"][mer] = sirius_df["most specific class"][0]
                            merged_df["level 5"][mer] = sirius_df["level 5"][0]
                        merged_df["superclass"][mer] = sirius_df["superclass"][0]
                        merged_df["class"][mer] = sirius_df["class"][0]
                        merged_df["subclass"][mer] = sirius_df["subclass"][0]
                        merged_df["ClassificationSource"][mer] = "CANOPUS"

        elif len(sirius_df) == 0:
            #print("NO Structures")
            merged_df["Formula"][mer] = np.nan
            merged_df["most specific class"][mer] = np.nan
            merged_df["level 5"][mer] = np.nan
            merged_df["superclass"][mer] = np.nan
            merged_df["class"][mer] = np.nan
            merged_df["subclass"][mer] = np.nan
            merged_df["ClassificationSource"][mer] = np.nan



        mbank_df = pd.read_csv(mbank_csv)
        if len(mbank_df) > 0:
            mbank_df = mbank_df.drop_duplicates("MBSMILES")
            mbank_df = mbank_df.dropna(subset=["MBSMILES"])

        gnps_df = pd.read_csv(gnps_csv)
        if len(gnps_df) > 0:
            gnps_df = gnps_df.drop_duplicates("GNPSSMILES")
            gnps_df = gnps_df.dropna(subset=["GNPSSMILES"])

        hmdb_df = pd.read_csv(hmdb_csv)
        if len(hmdb_df) > 0:
            hmdb_df = hmdb_df.drop_duplicates("HMDBSMILES")
            hmdb_df = hmdb_df.dropna(subset=["HMDBSMILES"])

            
            
            
        # 1 SGHM
        if (
            len(sirius_df) > 0
            and len(gnps_df) > 0
            and len(mbank_df) > 0
            and len(hmdb_df) > 0
        ):
            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(sirius_df["Source"])),
                *(list(gnps_df["Source"])),
                *(list(mbank_df["Source"])),
                *(list(hmdb_df["Source"])),
            ]

            rank_l2 = [
                *(list(sirius_df["rank_ids"])),
                *(list(gnps_df["rank_ids"])),
                *(list(mbank_df["rank_ids"])),
                *(list(hmdb_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(sirius_df["smiles"])),
                *(list(gnps_df["GNPSSMILES"])),
                *(list(mbank_df["MBSMILES"])),
                *(list(hmdb_df["HMDBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )
            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                mbank_df=mbank_df,
                gnps_df=gnps_df,
                hmdb_df=hmdb_df,
                Source="SGHM",
            )

            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )
            if max(candidates_with_counts["Count"]) == 4:
                sources_4(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 3:
                sources_3(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)

        # 2 SGM
        elif (
            len(sirius_df) > 0
            and len(gnps_df) > 0
            and len(mbank_df) > 0
            and len(hmdb_df) == 0
        ):

            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(sirius_df["Source"])),
                *(list(gnps_df["Source"])),
                *(list(mbank_df["Source"])),
            ]

            rank_l2 = [
                *(list(sirius_df["rank_ids"])),
                *(list(gnps_df["rank_ids"])),
                *(list(mbank_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(sirius_df["smiles"])),
                *(list(gnps_df["GNPSSMILES"])),
                *(list(mbank_df["MBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )
            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                mbank_df=mbank_df,
                gnps_df=gnps_df,
                # hmdb_df = hmdb_df,
                Source="SGM",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )

            if max(candidates_with_counts["Count"]) == 3:
                sources_3(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)

        # 3 SHM
        elif (
            len(sirius_df) > 0
            and len(gnps_df) == 0
            and len(mbank_df) > 0
            and len(hmdb_df) > 0
        ):

            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            # gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(sirius_df["Source"])),
                *(list(mbank_df["Source"])),
                *(list(hmdb_df["Source"])),
            ]

            rank_l2 = [
                *(list(sirius_df["rank_ids"])),
                *(list(mbank_df["rank_ids"])),
                *(list(hmdb_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(sirius_df["smiles"])),
                *(list(mbank_df["MBSMILES"])),
                *(list(hmdb_df["HMDBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )
            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                mbank_df=mbank_df,
                # gnps_df = gnps_df ,
                hmdb_df=hmdb_df,
                Source="SHM",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )

            if max(candidates_with_counts["Count"]) == 3:
                sources_3(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)

        # 4 SGH
        elif (
            len(sirius_df) > 0
            and len(gnps_df) > 0
            and len(mbank_df) == 0
            and len(hmdb_df) > 0
        ):
            # mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(sirius_df["Source"])),
                *(list(gnps_df["Source"])),
                *(list(hmdb_df["Source"])),
            ]

            rank_l2 = [
                *(list(sirius_df["rank_ids"])),
                *(list(gnps_df["rank_ids"])),
                *(list(hmdb_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(sirius_df["smiles"])),
                *(list(gnps_df["GNPSSMILES"])),
                *(list(hmdb_df["HMDBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:




                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )
            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                # mbank_df = mbank_df,
                gnps_df=gnps_df,
                hmdb_df=hmdb_df,
                Source="SGH",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )
            if max(candidates_with_counts["Count"]) == 3:
                sources_3(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)


        # 5 GHM
        elif (
            len(sirius_df) == 0
            and len(gnps_df) > 0
            and len(mbank_df) > 0
            and len(hmdb_df) > 0
        ):
            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            # sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
            # sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(gnps_df["Source"])),
                *(list(mbank_df["Source"])),
                *(list(hmdb_df["Source"])),
            ]

            rank_l2 = [
                *(list(gnps_df["rank_ids"])),
                *(list(mbank_df["rank_ids"])),
                *(list(hmdb_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(gnps_df["GNPSSMILES"])),
                *(list(mbank_df["MBSMILES"])),
                *(list(hmdb_df["HMDBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )
            one_candidate = one_candidate_selection(
                sm,
                # sirius_df = sirius_df,
                mbank_df=mbank_df,
                gnps_df=gnps_df,
                hmdb_df=hmdb_df,
                Source="GHM",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )

            if max(candidates_with_counts["Count"]) == 3:
                sources_3(candidates_with_counts, merged_df, mer, sirius_df=None)
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df=None)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df=None)

        # 6 SG
        elif (
            len(sirius_df) > 0
            and len(gnps_df) > 0
            and len(mbank_df) == 0
            and len(hmdb_df) == 0
        ):
            # mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(sirius_df["Source"])),
                *(list(gnps_df["Source"])),
            ]
            # ,*(list(mbank_df["Source"]))]

            rank_l2 = [
                *(list(sirius_df["rank_ids"])),
                *(list(gnps_df["rank_ids"])),
            ]
            # ,*(list(mbank_df["rank_ids"]))]

            smiles_l3 = [
                *(list(sirius_df["smiles"])),
                *(list(gnps_df["GNPSSMILES"])),
            ]
            # ,*(list(mbank_df["MBSMILES"]))]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                # mbank_df = mbank_df,
                gnps_df=gnps_df,
                # hmdb_df = hmdb_df,
                Source="SG",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )


            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)

        # 7 SH
        elif (
            len(sirius_df) > 0
            and len(gnps_df) == 0
            and len(mbank_df) == 0
            and len(hmdb_df) > 0
        ):
            # mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

            # gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(sirius_df["Source"])),
                *(list(hmdb_df["Source"])),
            ]

            rank_l2 = [
                *(list(sirius_df["rank_ids"])),
                *(list(hmdb_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(sirius_df["smiles"])),
                *(list(hmdb_df["HMDBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )
            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                # mbank_df = mbank_df,
                # gnps_df = gnps_df ,
                hmdb_df=hmdb_df,
                Source="SH",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )

            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)

        # 8 SM
        elif (
            len(sirius_df) > 0
            and len(gnps_df) == 0
            and len(mbank_df) > 0
            and len(hmdb_df) == 0
        ):
            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            # gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

            # hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(sirius_df["Source"])),
                *(list(mbank_df["Source"])),
            ]

            rank_l2 = [
                *(list(sirius_df["rank_ids"])),
                *(list(mbank_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(sirius_df["smiles"])),
                *(list(mbank_df["MBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                mbank_df=mbank_df,
                # gnps_df = gnps_df ,
                # hmdb_df = hmdb_df,
                Source="SM",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)

        # 9 GM
        elif (
            len(sirius_df) == 0
            and len(gnps_df) > 0
            and len(mbank_df) > 0
            and len(hmdb_df) == 0
        ):
            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            # hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

            # sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
            # sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(mbank_df["Source"])),
                *(list(gnps_df["Source"])),
            ]

            rank_l2 = [
                *(list(mbank_df["rank_ids"])),
                *(list(gnps_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(mbank_df["MBSMILES"])),
                *(list(gnps_df["GNPSSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                # sirius_df = sirius_df,
                mbank_df=mbank_df,
                gnps_df=gnps_df,
                # hmdb_df = hmdb_df,
                Source="GM",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df=None)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df=None)

        # 10 GH
        elif (
            len(sirius_df) == 0
            and len(gnps_df) > 0
            and len(mbank_df) == 0
            and len(hmdb_df) > 0
        ):
            # mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            # sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
            # sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(gnps_df["Source"])),
                *(list(hmdb_df["Source"])),
            ]

            rank_l2 = [
                *(list(gnps_df["rank_ids"])),
                *(list(hmdb_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(gnps_df["GNPSSMILES"])),
                *(list(hmdb_df["HMDBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )
            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )
            one_candidate = one_candidate_selection(
                sm,
                # sirius_df = sirius_df,
                # mbank_df = mbank_df,
                gnps_df=gnps_df,
                hmdb_df=hmdb_df,
                Source="GH",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )

            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df=None)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df=None)
        # 11 HM
        elif (
            len(sirius_df) == 0
            and len(gnps_df) == 0
            and len(mbank_df) > 0
            and len(hmdb_df) > 0
        ):
            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            # gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            # sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
            # sirius_df["Source"] = "SIRIUS"

            source_l1 = [
                *(list(mbank_df["Source"])),
                *(list(hmdb_df["Source"])),
            ]

            rank_l2 = [
                *(list(mbank_df["rank_ids"])),
                *(list(hmdb_df["rank_ids"])),
            ]

            smiles_l3 = [
                *(list(mbank_df["MBSMILES"])),
                *(list(hmdb_df["HMDBSMILES"])),
            ]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:

                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )
            one_candidate = one_candidate_selection(
                sm,
                # sirius_df = sirius_df,
                mbank_df=mbank_df,
                # gnps_df = gnps_df ,
                hmdb_df=hmdb_df,
                Source="HM",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )
            if max(candidates_with_counts["Count"]) == 2:
                sources_2(candidates_with_counts, merged_df, mer, sirius_df=None)
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df=None)

        # S
        elif (
            len(sirius_df) > 0
            and len(gnps_df) == 0
            and len(mbank_df) == 0
            and len(hmdb_df) == 0
        ):

            sirius_df["rank_ids"] = [
                "S_" + str(s) for s in sirius_df["rank"]
            ]
            sirius_df["Source"] = "SIRIUS"

            source_l1 = [*(list(sirius_df["Source"]))]

            rank_l2 = [*(list(sirius_df["rank_ids"]))]

            smiles_l3 = [*(list(sirius_df["smiles"]))]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)

            if df_edge is not None:

                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                sirius_df=sirius_df,
                # mbank_df = mbank_df,
                # gnps_df = gnps_df ,
                # hmdb_df = hmdb_df,
                Source="S",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )

            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df)
        # G
        elif (
            len(sirius_df) == 0
            and len(gnps_df) > 0
            and len(mbank_df) == 0
            and len(hmdb_df) == 0
        ):
            gnps_df["rank_ids"] = [
                "G_" + str(s + 1) for s in range(len(gnps_df))
            ]

            source_l1 = [*(list(gnps_df["Source"]))]

            rank_l2 = [*(list(gnps_df["rank_ids"]))]

            smiles_l3 = [*(list(gnps_df["GNPSSMILES"]))]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:

                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                # sirius_df = sirius_df,
                # mbank_df = mbank_df,
                gnps_df=gnps_df,
                # hmdb_df = hmdb_df,
                Source="G",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df=None)
        # M
        elif (
            len(sirius_df) == 0
            and len(gnps_df) == 0
            and len(mbank_df) > 0
            and len(hmdb_df) == 0
        ):
            mbank_df["rank_ids"] = [
                "M_" + str(s + 1) for s in range(len(mbank_df))
            ]

            source_l1 = [*(list(mbank_df["Source"]))]

            rank_l2 = [*(list(mbank_df["rank_ids"]))]

            smiles_l3 = [*(list(mbank_df["MBSMILES"]))]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:

                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                # sirius_df = sirius_df,
                mbank_df=mbank_df,
                # gnps_df = gnps_df ,
                # hmdb_df = hmdb_df,
                Source="M",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )
            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df=None)
        # H
        elif (
            len(sirius_df) == 0
            and len(gnps_df) == 0
            and len(mbank_df) == 0
            and len(hmdb_df) > 0
        ):
            hmdb_df["rank_ids"] = [
                "H_" + str(s + 1) for s in range(len(hmdb_df))
            ]

            source_l1 = [*(list(hmdb_df["Source"]))]

            rank_l2 = [*(list(hmdb_df["rank_ids"]))]

            smiles_l3 = [*(list(hmdb_df["HMDBSMILES"]))]

            sm = pd.DataFrame(
                list(zip(source_l1, rank_l2, smiles_l3)),
                columns=["Source", "ranks", "SMILES"],
            )

            df_edge = chemMN_CandidateSelection(sm)
            if df_edge is not None:


                df_edge.to_csv(
                    input_dir
                    + "/"
                    + entry
                    + "/"
                    + 'Candidate_Selection'
                    + "/"
                    + str(merged_df["premz"][mer])
                    + "_ChemMNedges.tsv",
                    sep="\t",
                )

            one_candidate = one_candidate_selection(
                sm,
                #                                                                         sirius_df = sirius_df,
                #                                                                         mbank_df = mbank_df,
                #                                                                         gnps_df = gnps_df ,
                hmdb_df=hmdb_df,
                Source="H",
            )
            candidates_with_counts = add_count_column(one_candidate)
            candidates_with_counts.to_csv(
                input_dir
                + "/"
                + entry
                + "/"
                + 'Candidate_Selection'
                + "/"
                + str(merged_df["premz"][mer])
                + "sorted_candidate_list.tsv",
                sep="\t",
            )

            if max(candidates_with_counts["Count"]) == 1:
                sources_1(candidates_with_counts, merged_df, mer, sirius_df=None)

    if standards:
        if not isNaN(merged_df["SMILES"][mer]):

            merged_df.loc[mer, "MSILevel"] = 1




#### code for formula and canopus classes
df_for_formula=pd.DataFrame(for_only_formula)
df_for_formula_n_canopus = pd.DataFrame(for_formula_canopus)



for m, row in merged_df.iterrows():
    for f, row in df_for_formula.iterrows():
        if m == df_for_formula["index"][f]:
            merged_df.loc[m, "Formula"] = df_for_formula["Formula"][f]
            merged_df.loc[m, "AnnotationSources"] = df_for_formula["AnnotationSources"][f]

    for c, row in df_for_formula_n_canopus.iterrows():
        if m == df_for_formula_n_canopus["index"][c]:
            merged_df.loc[m, "Formula"] = df_for_formula_n_canopus["Formula"][c]
            if "most specific class" in df_for_formula_n_canopus.columns:
                merged_df.loc[m, "most specific class"] = df_for_formula_n_canopus["subclass"][c]
            if "level 5" in df_for_formula_n_canopus.columns:
                merged_df.loc[m, "level 5"] = df_for_formula_n_canopus["subclass"][c]
            if "subclass" in df_for_formula_n_canopus.columns:
                merged_df.loc[m, "subclass"] = df_for_formula_n_canopus["subclass"][c]
                
            merged_df.loc[m, "class"] = df_for_formula_n_canopus["class"][c]
            merged_df.loc[m, "superclass"] = df_for_formula_n_canopus["superclass"][c]
            merged_df.loc[m, "ClassificationSource"] = df_for_formula_n_canopus["ClassificationSource"][c]
            merged_df.loc[m, "Formula"] = df_for_formula_n_canopus["Formula"][c]
            merged_df.loc[m, "AnnotationSources"] = df_for_formula_n_canopus["AnnotationSources"][c]



merged_df.to_csv(
    input_dir
    + "/"
    + entry
    + "/"
    + "mergedResults-with-one-Candidates.csv"
)
#print(merged_df)
merged_df = checkSMILES_validity(input_dir, resultcsv = input_dir
    + "/"
    + entry
    + "/"
    + "mergedResults-with-one-Candidates.csv")
merged_df.to_csv(
    input_dir
    + "/"
    + entry
    + "/"
    + "mergedResults-with-one-Candidates.csv"
)

